In [1]:
import re
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from bs4 import BeautifulSoup as bs

from importlib import reload
import carfax_scraper 


In [2]:
reload(carfax_scraper)
from carfax_scraper import pull_car_listings, build_car_df

In [3]:
plt.rcParams.update({'font.size': 14})

In [4]:
philly = {'url': 'https://www.carfax.com/Used-Cars-in-Philadelphia-PA_c4927',
          'zipcode': 19107,
          'name': 'philly'}
nyc = {'url': 'https://www.carfax.com/Used-Cars-in-Brooklyn-NY_c10456',
       'zipcode': 11222, 
       'name': 'nyc'}

location = philly

In [5]:
# option to work with previously saved results
soup_list = []
with open(location['name']+'_soup_progress.txt') as f:
    html_list = f.read().split('BREAKHERE')
html_list.remove('')

for page in html_list:
    soup = bs(page, parser='lxml')
    soup_list.append(soup)

In [6]:
soup_list = pull_car_listings(philly, 55)

/home/jules/Documents/metis/Regression/regression_project/carfax_scraper.py:60: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 60 of the file /home/jules/Documents/metis/Regression/regression_project/carfax_scraper.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = bs(html)


In [6]:
print('%i pages collected'%(len(soup_list)))

117 pages collected


In [8]:
cars = build_car_df(soup_list).set_index('index')
print('Shape with duplicates: (%i, %i)'%(cars.shape[0], cars.shape[1]))

cars = cars.drop_duplicates()
print('Shape without duplicates: (%i, %i)'%(cars.shape[0], cars.shape[1]))

Shape with duplicates: (2925, 14)
Shape without duplicates: (1183, 14)


In [9]:
def preprocessing(df, city=location['name']):
    # city: "philly" or other city name

    df = format_mileage(df)
    
    df['location'] = df['location'].apply(get_location)

    if city=='philly':
        df['location'] = df['location'].str.replace('Phila,', 'Philadelphia,')
        
    get_engine_info()

    df = drop_electric_cars(df)
    df = map_damage_values(df)
    df['num_owners'] = df['history'].apply(get_prev_owners)
    df['purchase_date'] = pd.to_datetime(df['history'].apply(get_purchase_date))

    # remove redundant column
    df.drop(columns='purchase_date', inplace=True)
    
    unique_options = get_options()
    format_options(df, unique_options)
    
    for col in ['price', 'year', 'mileage', 'cylinders', 'capacity', 'damage', 'service']:
        df.loc[:, col] = pd.to_numeric(df.loc[:, col])
    
    df = get_luxury_label(df)
    df = get_num_options(df)
    df = get_wheel_size(df)
    df = get_interior_options(df)
    
    return df

In [10]:
def format_mileage(df):
    df['mileage'] = df['mileage'].str.replace(',', '').str.replace(' miles', '')
    df.loc[df['mileage']=="N/A", 'mileage'] = np.nan
    return df


In [11]:
def get_location(x):
    match = re.search(r'.+(?=\s\()', x)
    if match:
        loc = match.group().split(',')
        formatted_location = ','.join([loc[0].title(), loc[1]])
        return formatted_location
    else:
        return(x) 
    

In [12]:
def get_engine_info():
    cars['cylinders'] = cars['engine'].apply(lambda x: x.split(' ')[0])
    cars['cylinders'] = cars['cylinders'].str.replace('Electric', '0').astype(int)
    cars['is_electric'] = cars['engine'].apply(lambda x: int(x == 'Electric'))
    cars['capacity'] = cars['engine'].apply(lambda x: x.split(' ')[2] if len(x.split(' ')) > 2 
                                            else 0).astype(float)

    


In [13]:
def drop_electric_cars(cars):
    df = cars[cars['is_electric'] == 0]
    df = df.drop('is_electric', axis=1)
    return df



In [14]:
def map_damage_values(df):
    damage_values = \
            {'No Accident or Damage Reported': 0,

             'Accident/No Damage': 1,
             'Very Minor Damage': 1,
             'Odometer Problem': 1,

             'Minor Damage': 2, 
             'Damage Reported': 2,
             'Accident Reported': 2,
             'Accident/Minor Damage': 2,
             'Vandalism Damage': 2,

             'Major Damage': 3}

    df['damage'] = df['damage'].map(damage_values)
    df['any_damage'] = (df['damage'] > 0).astype(int)
    
    return df

In [15]:
def get_prev_owners(x):
    match = re.search(r'\d+', x)
    if match:
        return int(match.group(0))
    else:
        return (1) # mode of prev owners




In [16]:
def get_purchase_date(x):
    # extracting original purchase date from owner history 
    dates = re.search(r'\d{2}/\d{2}/\d{2}', x)
    if dates:
        return(dates.group(0))
    else:
        return(np.nan)
    


In [17]:
def get_options():
    # Returns a list of the unique optional features from the whole dataset
    all_options = np.unique(', '.join(cars['options'].unique().astype(str)).split(', '))
    unique_options = np.unique([x.lstrip('and ') for x in all_options])[1:]
    return unique_options


In [18]:
def format_options(df, unique_options):
    for option in unique_options:
        df[option.replace(' ', '_').lower()] = df['options'].str.contains(option)
    return df

In [19]:
def get_luxury_label(df):
    response = requests.get('https://www.carfax.com/blog/luxury-car-brands')
    soup = bs(response.content)
    luxury_brands = [x.text for x in soup.find_all('h3')[1:-3]]

    df['luxury'] = df['make'].isin(luxury_brands).astype(int)
    return df

In [20]:
def get_num_options(df):
    df['num_options'] = df['options'].apply(lambda x: len(x.split(',')))
    return df

In [21]:
def get_wheel_size(df):
    df['wheel_size'] = df['options'].apply(lambda x: int(re.search(r'(\d+\s)\bInch Wheels\b', x).group(1))
                                           if re.search(r'Inch Wheels', x) else None)
    return df

In [22]:
def get_interior_options(df):
    
    df['leather'] = df['options'].str.contains('Leather').astype(int)
    df.drop(columns=df.filter(like='leather_').columns, inplace=True)

    df['navigation'] = df['options'].str.contains('Navigation').astype(int)
    df.drop(columns=df.filter(like='navigation_').columns, inplace=True)

    df['heated_extras'] = df['options'].str.contains('Heated').astype(int)
    df.drop(columns=df.filter(regex=r'heated_(?!extras)').columns, inplace=True)

    df['driver_support'] = df['options'].str.contains('Driver').astype(int)
    df.drop(columns=df.filter(regex=r'driver(?!_support)').columns, inplace=True)

    return df

In [23]:
cars = preprocessing(cars)

In [24]:
for col in ['price', 'year', 'mileage', 'cylinders', 'capacity', 'damage', 'service']:
    cars.loc[:, col] = pd.to_numeric(cars.loc[:, col])

In [25]:
cars.filter(like='wd').columns
cars.drop(columns=['4wd', 'awd'], inplace=True)

In [26]:
# combining several highly correlated columns 

if 'side_airbag' in cars.columns:
    cars['side_airbag'] = cars['side_airbag'] | cars['side_airbags']
else:
    cars['side_airbag'] = cars['side_airbags']
cars['spare_tire'] = cars['spare_tire'] | cars['compact_spare_tire']
cars['sync'] = cars['sync'] | cars['sync_3']
cars['bench_seat'] = cars['bench_seat'] | cars['rear_bench_seat']

cars.drop(['side_airbags', 'compact_spare_tire', 'sync_3', 'rear_bench_seat'], axis=1, inplace=True)

In [27]:
# change boolean columns to 0 or 1
cars[cars.select_dtypes('bool').columns] = cars[cars.select_dtypes('bool').columns].astype(int)

In [28]:
# optionally remove a small number of outliers -- none in current dataset

cars = cars[cars['price']<130000]

In [29]:
cars['log_price'] = cars['price'].apply(np.log)
cars['log_price_diff'] = cars['price'].apply(np.log)

In [30]:
nan_cols = cars.isna().sum()
nan_cols[nan_cols>0]

mileage         1
color           3
wheel_size    741
dtype: int64

In [31]:
# Fill missing values with mode or median values
cars['color'] = cars['color'].fillna(cars['color'].mode())
cars.loc[cars['color'].isna(), 'color'] = cars['color'].mode()[0]

cars['wheel_size'] = cars.groupby(['body_type'])['wheel_size'].apply(lambda x: x.fillna(x.median())).to_list()
cars['wheel_size'] = cars['wheel_size'].fillna(cars.wheel_size.median())


/home/jules/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [32]:
cars.to_csv(location['name']+'_dec2021.csv')